# Bore information

This example notebook describes how to retrieve data for a small region and epoch of interest, concatenate data from available sensors and calculate the annual mean NDVI values.  You can then select a location of interest based on the change between years, retrieve an NDVI time series for that location and select imagery from before and after the change event

In [1]:
#Import libraries
%pylab notebook
import datacube
from datacube.storage import masking
from datacube.storage.masking import mask_to_dict
from datacube.storage.storage import write_dataset_to_netcdf
from datacube.helpers import ga_pq_fuser

import pandas as pd
import xarray as xr
import numpy as np
import csv
import os
import datetime

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt
import matplotlib.dates
from IPython.display import display
import ipywidgets as widgets

import rasterio
import urllib
from pyproj import Proj, transform
from dateutil import tz
from_zone = tz.tzutc()
to_zone = tz.tzlocal()
dc = datacube.Datacube(app='dc-show changes in annual mean NDVI values')

Populating the interactive namespace from numpy and matplotlib


In [2]:
# %pylab notebook
# import datacube
# import xarray as xr
# from datacube.storage import masking
# from datacube.storage.masking import mask_to_dict
# from matplotlib.backends.backend_pdf import PdfPages
# from matplotlib import pyplot as plt
# import matplotlib.dates
# from IPython.display import display
# import ipywidgets as widgets
# import datetime
# import rasterio
# import pandas as pd
# import numpy as np
# from pyproj import Proj, transform
# from datacube.storage.storage import write_dataset_to_netcdf
# from dateutil import tz
# import csv
# import rasterio
# from_zone = tz.tzutc()
# to_zone = tz.tzlocal()
# from datacube.storage.storage import write_dataset_to_netcdf
# dc = datacube.Datacube(app='dc-show changes in annual mean NDVI values')

# Import borehole water level and location data

In [18]:
#Insert the ID code for the borehole of interest

bore_of_interest= 'RN010167'

In [19]:
#Read bore data. Please note the borehole data will need to be downloaded from the BoM groundwater explorer  
#and saved to your work space on the NCI

#import water level information from csv
all_data= pd.read_csv('/g/data/r78/Geohack/Input/4_bore/'+bore_of_interest+'.csv') ### User requirement, change directory

In [20]:
#Import or enter borehole coordinates

#Either...
#Enter coordinates in GDA94 as decimal degree format
# long= 134.061
# lat=-19.803

#Or...
#Read coordinates from a csv saved in your work space ### User requirement: hash out if entered coordinates above
coordinates_of_interest= pd.read_csv('/g/data/r78/Geohack/Input/4_bore/Bore_Detailed.csv')
coordinates_of_interest=coordinates_of_interest.set_index('Bore ID').loc[bore_of_interest] #read csv
bh_long=coordinates_of_interest.Longitude #return long
bh_lat=coordinates_of_interest.Latitude #return lat

#reproject
inProj = Proj(init='EPSG:4326')
outProj = Proj(init='EPSG:3577')
bh_x,bh_y = transform(inProj,outProj,bh_long,bh_lat)

print ("GDA 1994:",bh_long,bh_lat)
print ("Australian Albers:", bh_x,bh_y)

GDA 1994: 134.061 -19.803
Australian Albers: 215034.25619941473 -2123928.8430569237


# Complete Datacube query

In [6]:
#Spatiotemporal range and wavelengths/band of interest are defined

#Define temporal range
start_of_epoch = '1987-01-01'
end_of_epoch =  '2014-12-31'

#Define wavelengths/bands of interest
bands_of_interest = ['green',
                     'red', 
                     'nir',
                     'swir1']

#Define sensors of interest
sensors = ['ls8',
    'ls7',
    'ls5' ] 

#Create bounding box around the location of the stream gauge
lat_max = bh_lat + 0.05
lat_min = bh_lat - 0.05
lon_max = bh_long + 0.05
lon_min = bh_long - 0.05

#Create query
query = {'time': (start_of_epoch, end_of_epoch)}
query['x'] = (lon_min, lon_max)
query['y'] = (lat_max, lat_min)
query['crs'] = 'EPSG:4326'

In [7]:
print (query)

{'time': ('1987-01-01', '2014-12-31'), 'x': (134.011, 134.11100000000002), 'y': (-19.753, -19.853000000000002), 'crs': 'EPSG:4326'}


In [8]:
#Create cloud mask. This will define which pixel quality (PQ) artefacts are removed from the results.

mask_components = {'cloud_acca':'no_cloud',
'cloud_shadow_acca' :'no_cloud_shadow',
'cloud_shadow_fmask' : 'no_cloud_shadow',
'cloud_fmask' :'no_cloud',
'blue_saturated' : False,
'green_saturated' : False,
'red_saturated' : False,
'nir_saturated' : False,
'swir1_saturated' : False,
'swir2_saturated' : False,
'contiguous':True}

## Complete Datacube extraction
The extracted data is first filtered using the criteria in "mask_components". The cloudiness of the scenes is then tested, and any scenes that do not meet the given "cloud_threshold" are discarded

In [9]:
#Retrieve the data for each Landsat sensor

sensor_clean = {}
cloud_threshold = 0.90  ###User requirement: set cloud threshold. Default value is "0.90" or >90% image and <10% cloud cover
                        ###Scenes will not be retrieved that have less than the cloud threshold worth of image.

for sensor in sensors:
    #Load the NBAR and corresponding PQ
    sensor_nbar = dc.load(product= sensor+'_nbar_albers', group_by='solar_day', 
                          measurements = bands_of_interest,  **query)
    sensor_pq = dc.load(product= sensor+'_pq_albers', group_by='solar_day', 
                        fuse_func=ga_pq_fuser, **query)
    
    #Retrieve the projection information before masking/sorting
    crs = sensor_nbar.crs
    crswkt = sensor_nbar.crs.wkt
    affine = sensor_nbar.affine
    
    #Ensure there's PQ to go with the NBAR
    sensor_nbar = sensor_nbar.sel(time = sensor_pq.time)
    
    #Apply the PQ masks to the NBAR
    quality_mask = masking.make_mask(sensor_pq, **mask_components)
    good_data = quality_mask.pixelquality.loc[start_of_epoch:end_of_epoch]
    sensor_nbar2 = sensor_nbar.where(good_data)
    
    #Calculate the percentage cloud free for each scene
    cloud_free = masking.make_mask(sensor_pq, cloud_acca='no_cloud', cloud_fmask='no_cloud', 
                                   contiguous=True).pixelquality
    mostly_cloud_free = cloud_free.mean(dim=('x','y')) >= cloud_threshold
        
    #Discard data that does not meet the cloud_threshold
    mostly_good = sensor_nbar2.where(mostly_cloud_free).dropna(dim='time', how='all')
    mostly_good['product'] = ('time', numpy.repeat(sensor, mostly_good.time.size))    
    sensor_clean[sensor] = mostly_good

    print('loaded %s' % sensor) 
    

print ('complete')

loaded ls8
loaded ls7
loaded ls5
complete


In [10]:
# #Group PQ by solar day to avoid idiosyncracies of N/S overlap differences in PQ algorithm performance
# pq_albers_product = dc.index.products.get_by_name(sensors[0]+'_pq_albers')
# valid_bit = pq_albers_product.measurements['pixelquality']['flags_definition']['contiguous']['bits']

# def pq_fuser(dest, src):
#     valid_val = (1 << valid_bit)

#     no_data_dest_mask = ~(dest & valid_val).astype(bool)
#     np.copyto(dest, src, where=no_data_dest_mask)

#     both_data_mask = (valid_val & dest & src).astype(bool)
#     np.copyto(dest, src & dest, where=both_data_mask)

In [11]:
#Define which pixel quality artefacts you want removed from the results
# mask_components = {'cloud_acca':'no_cloud',
# 'cloud_shadow_acca' :'no_cloud_shadow',
# 'cloud_shadow_fmask' : 'no_cloud_shadow',
# 'cloud_fmask' :'no_cloud',
# 'blue_saturated' : False,
# 'green_saturated' : False,
# 'red_saturated' : False,
# 'nir_saturated' : False,
# 'swir1_saturated' : False,
# 'swir2_saturated' : False,
# 'contiguous':True}

In [12]:
# #Retrieve the NBAR and PQ data for sensor n
# sensor_clean = {}
# for sensor in sensors:
#     #Load the NBAR and corresponding PQ
#     sensor_nbar = dc.load(product= sensor+'_nbar_albers', group_by='solar_day', measurements = bands_of_interest,  **query)
#     sensor_pq = dc.load(product= sensor+'_pq_albers', group_by='solar_day', fuse_func=pq_fuser, **query)
#     #grab the projection info before masking/sorting
#     crs = sensor_nbar.crs
#     crswkt = sensor_nbar.crs.wkt
#     affine = sensor_nbar.affine
#     #This line is to make sure there's PQ to go with the NBAR
#     sensor_nbar = sensor_nbar.sel(time = sensor_pq.time)
#     #Apply the PQ masks to the NBAR
#     cloud_free = masking.make_mask(sensor_pq, **mask_components)
#     good_data = cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]
#     sensor_nbar = sensor_nbar.where(good_data)
#     sensor_clean[sensor] = sensor_nbar

In [13]:
#Check the output
sensor_nbar

<xarray.Dataset>
Dimensions:  (time: 567, x: 425, y: 452)
Coordinates:
  * time     (time) datetime64[ns] 1987-05-25T00:30:42.500000 ...
  * y        (y) float64 -2.118e+06 -2.118e+06 -2.118e+06 -2.118e+06 ...
  * x        (x) float64 2.097e+05 2.098e+05 2.098e+05 2.098e+05 2.098e+05 ...
Data variables:
    green    (time, y, x) int16 1057 1006 1057 1108 1057 1057 1108 1160 1160 ...
    red      (time, y, x) int16 1958 2004 2004 1958 1958 1912 1912 1819 1819 ...
    nir      (time, y, x) int16 2698 2754 2809 2809 2864 2754 2698 2698 2643 ...
    swir1    (time, y, x) int16 4116 4116 4116 3997 3957 3997 3957 3679 3719 ...
Attributes:
    crs:      EPSG:3577

In [14]:
#Concatenate data from different sensors together and sort so that observations are sorted by time rather
# than sensor
nbar_clean = xr.concat(sensor_clean.values(), dim='time')
time_sorted = nbar_clean.time.argsort()
nbar_clean = nbar_clean.isel(time=time_sorted)
nbar_clean.attrs['crs'] = crs
nbar_clean.attrs['affin|e'] = affine

## Process borehole water level information
Calculate the percentiles of water level height, and sort the dataframe according to date

In [21]:
all_data

,Bore ID,Date,Variable,Result (m),Quality Code
0,RN010167,1972-04-12 01:30,SWL,10.98,quality-A
1,RN010167,1972-04-19 00:30,SWL,10.98,quality-A
2,RN010167,1972-04-26,SWL,10.98,quality-A
3,RN010167,1972-05-03,SWL,10.97,quality-A
4,RN010167,1972-05-10 00:30,SWL,10.97,quality-A
5,RN010167,1972-05-17 00:30,SWL,10.96,quality-A
6,RN010167,1972-05-24 00:30,SWL,10.97,quality-A
7,RN010167,1972-06-28 00:30,SWL,10.97,quality-A
8,RN010167,1972-07-05 00:30,SWL,10.98,quality-A
9,RN010167,1972-07-17 00:30,SWL,11.00,quality-A


In [22]:
#Calculate "percentage exceedance" (perexc) for water level values
all_data= all_data.rename(columns={'Result (m)':'waterlevel', 'Date':'date'})  #Rename flow and date columns
all_data = all_data.sort_values('waterlevel', ascending=[False]) #Sort data by flow value
all_data['rank'] = np.arange(len(all_data)) + 1 #Create rank column and values
all_data['perexc'] = 100*(all_data['rank'])/(len(all_data)+1) #Calculate probability of each rank
all_data= all_data.sort_values(['date']) #Sort data by date
all_data['date']=pd.to_datetime(all_data['date'], format='%Y/%m/%d %H:%M:%S') #Change datetime format
all_data.date = all_data.date.map(lambda t: t.strftime('%Y-%m-%d')) #Change datetime format

# Return just the time and sensor product information from the Datacube extraction

In [23]:
product_time = nbar_clean[['time', 'product']].to_dataframe() #Add time and product to dataframe
product_time.index = product_time.index + pd.Timedelta(hours=10) #Roughly convert to local time
product_time.index = product_time.index.map(lambda t: t.strftime('%Y-%m-%d')) #Remove Hours/Minutes Seconds by formatting into a string

# Match the date of stream flow data to the date where satellite information exists

In [24]:
subset_data = pd.merge(all_data, product_time, left_on= 'date', 
                       right_index=True, how='inner') #Match dates and merge

In [25]:
subset_data['date']=pd.to_datetime(subset_data['date'], format='%Y/%m/%d %H:%M:%S') #Change datetime format

# Create interactive bore hydrograph plot

In [26]:
#Create date variables for axis on hydrograph plot to enable automatic axis scaling

subset_dates = subset_data['date'] #Define matched dates
all_dates=all_data['date']         #Define all dates

min_date=min(subset_data.date) #Create minimum date value to enable automatic axis scaling
max_date=max(subset_data.date) #Create maximum date value to enable automatic axis scaling

In [28]:
#Create flow variables for axis on hydrograph plot to enable automatic axis scaling

max_waterlevel=max(subset_data.waterlevel) #Define maximum flow for matched flow
min_waterlevel=min(subset_data.waterlevel) #Define minimum flow for matched flow

min_waterlevel= (min_waterlevel-(0.01*max_waterlevel)) #Create minimum flow value to enable automatic axis scaling
max_waterlevel= (max_waterlevel+(0.08*max_waterlevel)) #Create maximum flow value to enable automatic axis scaling

In [29]:
#create date variables
subset_dates = subset_data['date']
all_dates=all_data['date']

KeyError: 'Date'

In [ ]:
#Create interactive hydrograph

#create widget that enables interaction with hydrograph
w = widgets.HTML("Event information appears here when you click on the figure")
def callback(event):
    global time_int, discharge_int, devent
    devent = event
    time_int = event.xdata
    discharge_int= event.ydata
    time_int_ = time_int.astype(datetime64[D])
    w.value = 'time_int: {}'.format(time_int)

#Set up plot
fig = plt.figure(figsize=(11.69,8.27)) #Edit size of plot ###User should format as required
#fig = plt.figure(figsize=(10,10))  #Edit size of plot ###User should format if required
fig.canvas.mpl_connect('button_press_event', callback) #Plot setup
plt.title('Interactive bore hydrograph: '+bore_of_interest) #Plot title ###User should format if required
plt.show() #Plot setup
display(w) #Plot setup
plt.subplots_adjust(left=0.12, right=0.95, top=0.95, bottom=0.15) #Set border dimensions  ###User should format if required
fig.patch.set_facecolor('white') #Make border white ###User should format if required
fig.patch.set_alpha(0.99) #Make border white ###User should format if required


#plot
matplotlib.pyplot.plot_date(all_dates,all_data['Result (m)'], '-', label= 'SWL')
matplotlib.pyplot.plot_date(subset_dates, subset_data['Result (m)'], 'ro', label='SWL values with satellite imagery')

#axis details
firstyear = '1986-01-01'
lastyear = '2014-12-30'
plt.axis([firstyear , lastyear ,5, 11], 'tight')
plt.xticks(rotation=45,size=10)
plt.ylabel('SWL (m)')
plt.xlabel('Date')
plt.legend(edgecolor ='none', ncol=2, loc=9)
plt.subplots_adjust(left=0.12, right=0.95, top=0.95, bottom=0.15)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.99)



In [ ]:
#Create interactive hydrograph

#create widget that enables interaction with hydrograph
w = widgets.HTML("Event information appears here when you click on the figure")
def callback(event):
    global time_int, discharge_int, devent
    devent = event
    time_int = event.xdata
    discharge_int= event.ydata
    time_int_ = time_int.astype(datetime64[D])
    w.value = 'time_int: {}'.format(time_int)

#plot setup 
#fig = plt.figure(figsize=(10,10))
fig = plt.figure(figsize=(11.69,8.27))
fig.canvas.mpl_connect('button_press_event', callback)
plt.title('Interactive bore hydrograph: ' + bore_of_interest )
plt.show()
display(w)

#plot
matplotlib.pyplot.plot_date(all_dates,all_data['Result (m)'], '-', label= 'SWL')
matplotlib.pyplot.plot_date(subset_dates, subset_data['Result (m)'], 'ro', label='SWL values with satellite imagery')

#axis details
firstyear = '1986-01-01'
lastyear = '2014-12-30'
plt.axis([firstyear , lastyear ,5, 11], 'tight')
plt.xticks(rotation=45,size=10)
plt.ylabel('SWL (m)')
plt.xlabel('Date')
plt.legend(edgecolor ='none', ncol=2, loc=9)
plt.subplots_adjust(left=0.12, right=0.95, top=0.95, bottom=0.15)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.99)



In [ ]:
 #save figure
%cd /g/data/r78/Geohack/Input/4_bore/
plt.savefig('Hydrograph_'+bore_of_interest+'_all.jpg')

In [ ]:
print 'Discharge: ' + str(discharge_int) + ' m3'
print 'Date as int: ' + str(time_int)

In [ ]:
time_slice = matplotlib.dates.num2date(time_int).date()
time_slice=str(time_slice)
time_slice= pd.to_datetime(time_slice, format='%Y-%m-%d')
subset_data['Date'] = pd.to_datetime(subset_data['Date'], format='%Y-%m-%d')
subset_data['difference']=(subset_data['Date'] - time_slice).abs()
time_slice=subset_data.ix[argsort(subset_data['difference'])].Date
time_slice= (list(time_slice)[0])
time_slice= str(time_slice)
time_slice=datetime.datetime.strptime(time_slice,'%Y-%m-%d  %H:%M:%S')
time_slice_actual=time_slice
time_slice_t1=time_slice_actual+datetime.timedelta(days=-3)
time_slice_t2=time_slice_actual+datetime.timedelta(days=3)

# #Discharge
# # discharge_title= subset_data.ix[argsort(subset_data['difference'])].'Result (m)'
# # discharge_title= (list(discharge_title)[0])
# # discharge_title= str(discharge_title)
# # discharge_title2= float(discharge_title)
# # discharge_title2=str("{0:.2f}".format(discharge_title2))

# #Percentage exceedance
# perexc_title= subset_data.ix[argsort(subset_data['difference'])].perexc
# perexc_title= (list(perexc_title)[0])
# perexc_title= str(perexc_title)
# perexc_title2= float(perexc_title)
# perexc_title2=str("{0:.2f}".format(perexc_title2))


#Satellite
satellite_type=subset_data.ix[argsort(subset_data['difference'])]
satellite_type=satellite_type['product']
satellite_type= (list(satellite_type)[0])
satellite_type= str(satellite_type)
satellite_type=  satellite_type.replace('pq','nbar')


print 'Time 1:' +str(time_slice_t1)
print 'Actual observation date: ' +str(time_slice_actual)
print 'Time 2: ' +str(time_slice_t2)
# print 'Discharge: ' +str(discharge_title2) +' m3'
#print 'Percent exceedance: '+ str(perexc_title2) + '%'
print 'Product: '+ str(satellite_type)


In [ ]:
#This will work for time series that correspond with sensor1
rgb = nbar_clean.sel(time =time_slice, method = 'nearest').to_array(dim='color').sel(color=['swir1', 'nir', 'green']).transpose('y', 'x', 'color')
fake_saturation = 6000
clipped_visible = rgb.where(rgb<fake_saturation).fillna(fake_saturation)
max_val = clipped_visible.max(['y', 'x'])
scaled2 = (clipped_visible / max_val)

In [ ]:
#This image shows the time slice of choice and the location of the time series 
fig = plt.figure(figsize =(8,8))
#plt.title('Gauge: '+gauge_of_interest +'    Date: '+str(time_slice_actual)[0:-9]  + '    Discharge: ' + discharge_title2+' $m^3$ $day^{-1}$' +
#          '   Percentage exceedance: '+ str(perexc_title2) + '%', size=10)
plt.scatter(x = [bh_x], y = [bh_y], c= 'r', marker = 'o', s=150)
plt.imshow(scaled2, interpolation = 'nearest',
           extent=[scaled2.coords['x'].min(), scaled2.coords['x'].max(), 
                  scaled2.coords['y'].min(), scaled2.coords['y'].max()])
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.99)
#remove axis 
plt.axis('off')

plt.show()

In [ ]:
#save figure
%cd /g/data/r78/Geohack/Input/4_bore/
plt.savefig('Hydrograph_'+ str(time_slice_actual)[0:-9] + '_' + bore_of_interest+'.jpg')

# Creation of large image

In [ ]:
start_of_epoch = time_slice_t1.strftime("%Y %m, %d")
end_of_epoch = time_slice_t2.strftime("%Y %m, %d")

query2 = {
    'time': (start_of_epoch, end_of_epoch),
}
bands_of_interest = [#'blue',
                     'green',
                     #'red', 
                     'nir',
                     'swir1', 
                     #'swir2'
                     ]

#Define sensors of interest, # out sensors that aren't relevant for the time period
lat_max = bh_lat+ 0.6
lat_min = bh_lat- 0.6
lon_max = bh_long+ 0.15
lon_min = bh_long- 0.8

query2['x'] = (lon_min, lon_max)
query2['y'] = (lat_max, lat_min)
query2['crs'] = 'EPSG:4326'

In [ ]:
"""
    'ls8', #May 2013 to present
    'ls7', #1999 to April 2003 (after this it'll have venetian blind artefacts caused by SLC off)
    'ls5' #1987 to 1999 and then from April 2003 to 2011, """
    
image_of_interest = dc.load(product= satellite_type, group_by='solar_day', measurements = bands_of_interest,  **query2)


In [ ]:
image_of_interest

In [ ]:
rgb = image_of_interest.to_array(dim='color').sel(color=['swir1','nir', 'green']).squeeze().transpose('y', 'x', 'color')

fake_saturation = 6000
clipped_visible = rgb.where(rgb<fake_saturation).fillna(fake_saturation)
max_val = clipped_visible.max(['y', 'x'])
scaled = (clipped_visible / max_val)

In [ ]:
##Create large image

#fig = plt.figure(figsize =(21,21))
fig = plt.figure(figsize =(10,10))

#plt.title('Date: '+str(time_slice_actual)[0:-9]  + '    Flow: ' + discharge_title2+' $m^3$ $day^{-1}$',size=26)

#plt.title('Gauge: '+gauge_of_interest +'    Date: '+str(time_slice_actual)[0:-9]  + '    Discharge: ' + discharge_title2+' $m^3$ $day^{-1}$' +
#          '   Percentage exceedance: '+ str(perexc_title2) + '%', size=22)

#plot imagery and add stream gauging location as marker
plt.scatter(x = [bh_x], y = [bh_y], c= 'r', marker = 'o', s=100)

plt.imshow(scaled, interpolation = 'nearest',
           extent=[scaled.coords['x'].min(), scaled.coords['x'].max(), 
                  scaled.coords['y'].min(), scaled.coords['y'].max()])
#reformat
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.99)

#remove axis 
plt.axis('off')



In [ ]:
#save figure
%cd /g/data/r78/Geohack/Input/4_bore/
plt.savefig('Hydrograph_'+ str(time_slice_actual)[0:-9] +'_'+ bore_of_interest+'_large.jpg')

# Save as netcdf

In [ ]:
#get the original nbar dataset attributes (crs)
#set up variable attributes to hold the attributes
attrs = image_of_interest
#get the band info
bands = attrs.data_vars.keys()
print bands
for i in bands:
    #drop band data, retaining just the attributes
    attrs =attrs.drop(i)
#set up new variable called ndvi_var, and assign attributes to it in a dictionary
image_var = {'scaled':''}
image_output = attrs.assign(**image_var)
image_output['scaled'] = scaled
print image_output
image_output2 = image_output.scaled.to_dataset(dim='color')
#print image_output
image_output2.attrs['crs'] = image_output.crs


In [ ]:
## create net cdf
outfile = '/g/data/r78/ext547/Output/netcdf/'+ str(time_slice_actual)[0:-9] +'.nc'
write_dataset_to_netcdf(image_output2,  variable_params={'scaled': {'zlib':True}}, filename=outfile)
print 'wrote: '+outfile+' to netcdf'

In [ ]:
# DEFAULT_PROFILE = {
#    'blockxsize': 256,
#    'blockysize': 256,
#    'compress': 'lzw',
#    'driver': 'GTiff',
#    'interleave': 'band',
#    'nodata': 0.0,
#    'photometric': 'RGBA',
#    'tiled': True}


# #this function definition comes from CK's Principal_Component_Analysis_AGDC_looped notebook, and before that from AGDC recipes.
# def write_geotiff(filename, dataset, time_index=None, profile_override=None):
#     """
#     Write an xarray dataset to a geotiff
#     :attr bands: ordered list of dataset names
#     :attr time_index: time index to write to file
#     :attr dataset: xarray dataset containing multiple bands to write to file
#     :attr profile_override: option dict, overrides rasterio file creation options.
#     """
#     profile_override = profile_override or {}

#     dtypes = {val.dtype for val in dataset.data_vars.values()}
#     assert len(dtypes) == 1  # Check for multiple dtypes

# #     profile = DEFAULT_PROFILE.copy()
# #     profile.update({
# #         'width': dataset.dims['x'],
# #         'height': dataset.dims['y'],
#         #'affine': dataset.affine, #changed following line 17/03/17 due to changes in AGDC I think
#         'affine': dataset.affine,
#         'crs': dataset.crs.crs_str,
#         #'crs': dataset.crs,
#         'count': len(dataset.data_vars),
#         'dtype': str(dtypes.pop())
#     })
#     profile.update(profile_override)

#     with rasterio.open(filename, 'w', **profile) as dest:
#         for bandnum, data in enumerate(dataset.data_vars.values(), start=1):
#             #dest.write(data.isel(time=time_index).data, bandnum)
#             dest.write(data, bandnum)
#             print ('Done')

In [ ]:
# outfile = '/g/data/r78/ext547/Working/' + 'test' + '.tiff'
# image_output2.attrs['crs'] = image_output.crs
# write_geotiff(outfile, image_output2)
# print 'tiff writing complete'

In [ ]:

#est_output2.y[1] - test_output2.y[0]

In [ ]:
#%cd /g/data/r78/ext547/Working/
#!ls

In [ ]:
#!gdalinfo test.tiff

In [ ]:
# def pq_fuser(dest, src):
#     valid_bit = 8
#     valid_val = (1 << valid_bit)
#     no_data_dest_mask = ~(dest & valid_val).astype(bool)
#     np.copyto(dest, src, where=no_data_dest_mask)
#     both_data_mask = (valid_val & dest & src).astype(bool)
#     np.copyto(dest, src & dest, where=both_data_mask)
# indexers = {'time':('1991', '1992'), 'x':(lon_min, lon_max),   'y':(lat_min, lat_max), 'group_by':'solar_day'}
# data = dc.load(product='ls5_nbar_albers', **indexers)
# pq = dc.load(product='ls5_pq_albers', fuse_func=pq_fuser, **indexers)
# mask_clear = pq['pixelquality'] & 15871 == 15871    # 15871 - This is a cloud free bits that includes land and sea bits
# data = data.where(mask_clear)

# from datacube.storage import masking
# pq = dc.load(product='ls5_pq_albers', x=(lon_min, lon_max), y=(lat_min, lat_max), 
#              time=('1991', '1992'), group_by='solar_day')
# # Make a mask for clouds
# cloud_free = masking.make_mask(pq, cloud_acca='no_cloud', cloud_fmask='no_cloud', contiguous=True).pixelquality
# # Find where at least 75% of the image is cloud free
# mostly_cloud_free = cloud_free.mean(dim=('x','y')) > 0.75
# # return only the NDVI times where there are no clouds
# # dropna - return object with labels on given axis omitted where any/all the data are missing
# #mostly_good_ndvi = ndvi.where(mostly_cloud_free).dropna('time', how='all')
# #mostly_cloud_free.plot(col='time', col_wrap=3)
# mostly_cloud_free = cloud_free.sum(dim=('x','y')) > (0.75 * data.green.size / data.time.size)
# test = data.where(mostly_cloud_free).dropna('time', how='all')

# print test.isel(time =1)